In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import regularizers
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler


ImportError: cannot import name 'Dense' from 'keras' (c:\Users\chris\VS Code Projects\Stealth-EMG-SVM\.venv\Lib\site-packages\keras\__init__.py)

In [ ]:
#---CONFIG FRAME---#
train_file = [
    "parsed_emg/subject_1/trial_11(right).csv",
    "parsed_emg/subject_1/trial_3(right).csv",
    "parsed_emg/subject_1/trial_5(right).csv",
    "parsed_emg/subject_1/trial_7(right).csv",
    "parsed_emg/subject_1/trial_9(right).csv"
]

test_file = "parsed_emg/subject_1/trial_1(right).csv"

label_col = 5
batch_size = 64
learning_rate = 0.001
epochs = 1
scaler = StandardScaler()

#---Data Loader---#
def load_csv(path):
    df = pd.read_csv(path, skiprows=1)
    df = df.drop(columns=df.columns[0], axis=1)
    X = df.iloc[:, 1:5].values.astype("float32")
    y = df.iloc[:, label_col].values.astype("int32")
    return X, y



In [59]:
# --- Prepare the training dataset ---
training_datasets = []
for f in train_file:
    X_tr, y_tr = load_csv(f)       # X_tr: shape (N, input_dim), y_tr: integer labels [0,1,2]
    ds = tf.data.Dataset.from_tensor_slices((X_tr, y_tr))
    training_datasets.append(ds)

# Concatenate all per-file datasets
training_ds = training_datasets[0]
for ds in training_datasets[1:]:
    training_ds = training_ds.concatenate(ds)

# Shuffle, batch, and prefetch
training_ds = (
    training_ds
    .shuffle(buffer_size=5000)
    .batch(batch_size)
)

# Load the test data
X_test, y_test = load_csv(test_file)

# --- Build the model ---
num_classes = 3                # three target categories
input_dim  = X_tr.shape[1]
print(input_dim)    # infer number of input features

model = tf.keras.Sequential([
    layers.Input(shape=(input_dim,)),
    # Hidden layer for non-linearity
    layers.Dense(
        12,
        activation="relu",
        kernel_regularizer=regularizers.l2(1e-3)
    ),
    layers.Dropout(0.3),
    layers.Dense(
        8,
        activation="relu",
        kernel_regularizer=regularizers.l2(1e-3)
    ),
    # Output layer with softmax for 3-way classification
    layers.Dense(
        num_classes,
        activation="softmax",
        kernel_regularizer=regularizers.l2(1e-3)
    ),
])

# --- Compile with sparse categorical crossentropy for integer labels ---
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss="sparse_categorical_crossentropy",  # use this when y labels are integers
    metrics=["accuracy"],
)

# --- Train ---
model.fit(training_ds, epochs=epochs)

# preds = model.predict(X_test)
# print(np.argmax(preds[100000:100010], axis=1))  # predicted classes
# print(y_test[100000:100010])

# --- Evaluate ---
test_loss, test_acc = model.evaluate(X_test, y_test, batch_size=batch_size)
print(f"\nTest loss: {test_loss:.4f} — Test accuracy: {test_acc:.4f}")


4
36628/36628 ━━━━━━━━━━━━━━━━━━━━ 24s 635us/step - accuracy: 0.9834 - loss: 0.0801
1883/1883 ━━━━━━━━━━━━━━━━━━━━ 1s 561us/step - accuracy: 0.0294 - loss: 9.6172

Test loss: 7.3963 — Test accuracy: 0.2316
